# Configuration Interaction with Single Excitations

In this project, you will learn how to write a restricted configuration interaction with single excitations (CIS) code using the Python interface to the Psi4 electronic structure package. Most of Psi4's core libraries (integral generation, SCF solver, etc.) are exposed to Python, which facilitates the development of short, easy to understand codes for complex electronic structure methods. For example, you will soon find that it is possible to write a working CIS code in Python in less than 100 lines!


At the Hartree-Fock level of theory, the <i>N</i>-electron wave function is approximated as an antisymmetrized product of <i>N</i> one-electron functions called molecular orbitals (MOs) (a Slater determinant). The simplest representation of an excited-state wave function would be a single Slater determinant comprised of a different set of molecular
orbitals, but we can obtain a slightly better description of the excited state by expressing it as a linear combination of Slater determinants that differ by the ground-state configuration by one electron.  What we are describing here is a configuration interaction with single excitations (CIS) wave function; the CIS wave function for the <i>n</i><sup>th</sup> excited state can be expressed as

$$|\psi(n)\rangle = \sum_{ia} c_i^a(n) |\phi_i^a\rangle$$

Here, the symbol <i>&phi;<sub>i</sub><sup>a</sup></i> represents a Slater determinant that is singly excited relative to the reference determinant, and the indices <i>i</i> and <i>a</i> represent a spin orbitals that are occupied and unoccupied (virtual) in the reference, respectively.

The expansion coefficients and energy of the excited state can be determined by testing the Schrödinger equation with one of the singly-excited configurations to obtain

$$\sum_{jb} \langle \phi_i^a | \hat{H} | \phi_j^b \rangle c_j^b(n) = E(n) c_i^a(n)$$

We can form similar expressions for all excited states, and, taken together, these expressions become an eigenvalue equation

$${\bf H}{\bf c}(n) = E_n {\bf c}(n)$$

where <b>H</b> is the matrix representation of the Hamiltonian in the basis of singly-excited determinants.  The elements of this matrix are given by

$$H_{ia,jb} = F_{ab}\delta_{ij}-F_{ij}\delta_{ab} + \langle aj ||ib\rangle$$

where $F$ represents the Fock matrix, and $\langle aj || ib \rangle$ represents an antisymmetrized electron-repulsion integral in physicists' notation.  Recall that the Fock matrix for restricted Hartree-Fock theory is diagonal in the MO basis, so this expression simplifies sligthly:

$$H_{ia,jb} = (\epsilon_a - \epsilon_i) \delta_{ij}\delta_{ab} + \langle aj ||ib\rangle$$

where $\epsilon_i$ and $\epsilon_a$ represent occupied and virtual orbital energies, respectively.


